## Build Graph

In [ ]:
import logging
from llm.factory import LLMInterface
from llm.embedding import get_text_embedding
from knowledge_graph.knowledge import KnowledgeBuilder
from knowledge_graph.graph_builder import KnowledgeGraphBuilder

llm_client = LLMInterface("openai_like", "qwen3-32b")
kb_builder = KnowledgeBuilder()
graph_builder = KnowledgeGraphBuilder(llm_client, get_text_embedding)

# Initialize logging module with a basic configuration for console output
logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(levelname)s - %(filename)s:%(lineno)d: %(message)s'
)

In [ ]:
import json
import os

# Define the path to the JSON configuration file
config_file_path = 'docs/pdf_metadata.json'

# Variable to store the loaded data
loaded_docs = []

# Read the JSON configuration file
try:
    with open(config_file_path, 'r', encoding='utf-8') as f:
        loaded_docs = json.load(f)
    print(f"Successfully loaded configuration from: {config_file_path}")
except FileNotFoundError:
    print(f"Error: Configuration file not found at '{config_file_path}'")
except json.JSONDecodeError:
    print(f"Error: Could not decode JSON from file '{config_file_path}'. Check file format.")
except Exception as e:
    print(f"An unexpected error occurred while reading the file: {e}")

if len(loaded_docs) > 0:
    print("\nExample: Accessing first document data:")
    print(loaded_docs[0])
else:
    print("\nConfiguration file is empty.")


client_name_list = set([doc['client_name'] for doc in loaded_docs])
client_docs = {}
for client_name in client_name_list:
    client_docs[client_name] = []
    for doc in loaded_docs:
        if doc['client_name'] == client_name:
            client_docs[client_name].append({
                'path': doc['path'],  # required
                'doc_link': doc['web_view_link'], # required
                'topic_name': f"Customer Tracking for {doc['client_name']}",
                'client_name': doc['client_name'],
                'created_time': doc['created_time'],
                'modified_time': doc['modified_time'],
                'mime_type': doc['mime_type']
            })
    print(f"Client: {client_name}, Number of documents: {len(client_docs[client_name])}")


### Python Client Example

In [ ]:
client_name = "Airbnb"
docs = client_docs[client_name]

topic_name = f"Customer Tracking for {client_name}"

print("step 1: upload docs to knowledge base")
topic_docs = {}
for doc in docs:
    file_path = doc['path']
    try:
        res = kb_builder.extract_knowledge(
            file_path, 
            doc
        )
        if res['status'] == 'success':
            topic_docs[res['source_id']] = {
                "source_id": res['source_id'],
                "source_name": res['source_name'],
                "source_content": res['source_content'],
                "source_link": res['source_link'],
            } 
        else:
            print(f"process index {file_path} failed, {res['error']}", exc_info=True)

    except Exception as e:
        logging.error(f"process index {file_path} failed, {e}", exc_info=True)

topic_docs

In [ ]:
topic_name = f"Customer {client_name} Tracking"

print("step 2: add to graph")
result = graph_builder.build_iterative_knowledge_graph(topic_name, list(topic_docs.values()))
print(result)

### Restful API Example

In [ ]:
client_name = "Apple"
docs = client_docs[client_name]
docs

In [ ]:
os.getenv("GRAPH_DATABASE_URI")

In [ ]:
import requests

topic_name = f"Customer Tracking for {client_name}"
url = "http://192.168.206.252:23333/api/v1/knowledge/upload"

files = []
links = []
for doc in docs:
    files.append(('files', (doc["path"].split('/')[-1], open(doc["path"], 'rb'), 'application/pdf')))
    links.append(doc["doc_link"])

data = {
    'links': links,
    'topic_name': topic_name,
    'database_uri': os.getenv("GRAPH_DATABASE_URI")
}
response = requests.post(url, files=files, data=data)

print(response.status_code)
print(response.json())


In [ ]:
import requests

database_uri = os.getenv("GRAPH_DATABASE_URI")

# Call the trigger-processing API to start processing uploaded all documents for a topic
url = "http://192.168.206.252:23333/api/v1/knowledge/trigger-processing"
data = {
    "topic_name": topic_name,
    "database_uri": database_uri
}

response = requests.post(url, data=data)
print(response.status_code)
print(response.json())

## Query Graph

### Vector Similarity based Search

In [ ]:
from knowledge_graph.query import search_relationships_by_vector_similarity, query_topic_graph

res = search_relationships_by_vector_similarity("What are Apple concerning about?", similarity_threshold=0.4, top_k=20)
context = ""
entities = set()
relationships = []

for index, row in res.iterrows():
    entities.add(f"{row['source_entity']} {row['source_entity_description']}")
    entities.add(f"{row['target_entity']} {row['target_entity_description']}")
    relationships.append(f"{row['source_entity']} {row['relationship_desc']} {row['target_entity']}")

context = "Entities:\n" + "\n".join(entities) + "\n\nRelationships:\n" + "\n".join(relationships)

print(context)

In [ ]:
from llm.factory import LLMInterface

llm_client = LLMInterface("bedrock", "us.anthropic.claude-3-7-sonnet-20250219-v1:0")
response =llm_client.generate(f"""Given the following context
<context>
{context}
</context>
answer the question: "What are Apple concerning about?"
""")
print(response)

### Topic Graph

In [ ]:
import os

from knowledge_graph.query import query_topic_graph

subgraph = query_topic_graph("Customer Tracking for Roblox", os.getenv("GRAPH_DATABASE_URI"))
subgraph

In [ ]:
len(subgraph["relationships"])

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib.patches import FancyBboxPatch
import numpy as np
from collections import defaultdict
import textwrap

def visualize_knowledge_graph(subgraph_data, max_nodes=50, node_size_range=(1000, 3000)):
    """
    Visualize knowledge graph with enhanced styling
    """
    # Create directed graph
    G = nx.DiGraph()
    
    # Extract entities and relationships
    entities = subgraph_data.get('entities', [])
    relationships = subgraph_data.get('relationships', [])
    
    # Limit number of nodes for better visualization
    entities = entities[:max_nodes]
    
    # Create entity mapping
    entity_map = {entity['name']: entity for entity in entities}
    entity_names = set(entity_map.keys())
    
    # Add nodes
    for entity in entities:
        # Parse attributes
        import json
        try:
            attrs = json.loads(entity['attributes'])
            category = attrs.get('category', 'Unknown')
            entity_type = attrs.get('entity_type', 'Unknown')
        except:
            category = 'Unknown'
            entity_type = 'Unknown'
            
        G.add_node(entity['name'], 
                  category=category,
                  entity_type=entity_type,
                  description=entity['description'])
    
    # Add edges (only for entities that exist in our limited set)
    edge_count = 0
    for relationship in relationships:
        source = relationship['source_entity']
        target = relationship['target_entity']
        
        if source in entity_names and target in entity_names and edge_count < 100:
            G.add_edge(source, target, 
                      relationship=relationship['relationship_desc'][:100] + "..." if len(relationship['relationship_desc']) > 100 else relationship['relationship_desc'])
            edge_count += 1
    
    # Create figure with larger size
    plt.figure(figsize=(20, 16))
    
    # Use spring layout with better parameters
    pos = nx.spring_layout(G, k=3, iterations=50, seed=42)
    
    # Color mapping for categories
    categories = list(set([G.nodes[node].get('entity_type', 'Unknown') for node in G.nodes()]))
    colors = plt.cm.Set3(np.linspace(0, 1, len(categories)))
    category_colors = dict(zip(categories, colors))
    
    # Node colors based on category
    node_colors = [category_colors.get(G.nodes[node].get('entity_type', 'Unknown'), 'lightblue') for node in G.nodes()]
    
    # Node sizes based on degree (connectivity)
    degrees = dict(G.degree())
    min_size, max_size = node_size_range
    if degrees:
        max_degree = max(degrees.values()) if degrees.values() else 1
        min_degree = min(degrees.values()) if degrees.values() else 1
        degree_range = max_degree - min_degree if max_degree > min_degree else 1
        node_sizes = [min_size + (max_size - min_size) * (degrees.get(node, 1) - min_degree) / degree_range for node in G.nodes()]
    else:
        node_sizes = [min_size] * len(G.nodes())
    
    # Draw edges first (so they appear behind nodes)
    nx.draw_networkx_edges(G, pos, edge_color='gray', alpha=0.6, width=1, arrows=True, 
                          arrowsize=20, arrowstyle='->', connectionstyle='arc3,rad=0.1')
    
    # Draw nodes
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=node_sizes, 
                          alpha=0.8, linewidths=2, edgecolors='black')
    
    # Add labels with better positioning
    label_pos = {}
    for node, (x, y) in pos.items():
        # Wrap long labels
        wrapped_label = '\n'.join(textwrap.wrap(node, width=15))
        label_pos[node] = wrapped_label
    
    nx.draw_networkx_labels(G, pos, labels=label_pos, font_size=8, font_weight='bold')
    
    # Create legend for categories
    legend_elements = [plt.Rectangle((0,0),1,1, facecolor=category_colors[cat], 
                                   edgecolor='black', label=cat) for cat in categories[:10]]  # Limit legend items
    
    plt.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1, 1), 
              title="Entity Categories", title_fontsize=12)
    
    # Set title and remove axes
    plt.title(f"Apple Knowledge Graph Visualization\n({len(G.nodes())} entities, {len(G.edges())} relationships)", 
              fontsize=16, fontweight='bold', pad=20)
    plt.axis('off')
    
    # Adjust layout to prevent legend cutoff
    plt.tight_layout()
    
    # Add graph statistics
    stats_text = f"""Graph Statistics:
    • Nodes: {len(G.nodes())}
    • Edges: {len(G.edges())}
    • Avg Degree: {np.mean(list(degrees.values())):.2f}
    • Max Degree: {max(degrees.values()) if degrees else 0}
    • Connected Components: {nx.number_weakly_connected_components(G)}"""
    
    plt.figtext(0.02, 0.02, stats_text, fontsize=10, 
               bbox=dict(boxstyle="round,pad=0.5", facecolor="lightgray", alpha=0.8))
    
    plt.show()
    
    # Print some analysis
    print(f"\\n=== Knowledge Graph Analysis ===")
    print(f"Total entities: {len(entities)}")
    print(f"Total relationships: {len(relationships)}")
    print(f"Displayed entities: {len(G.nodes())}")
    print(f"Displayed relationships: {len(G.edges())}")
    
    if degrees:
        # Find most connected entities
        top_entities = sorted(degrees.items(), key=lambda x: x[1], reverse=True)[:5]
        print(f"\\nMost connected entities:")
        for entity, degree in top_entities:
            print(f"  • {entity}: {degree} connections")
    
    # Category distribution
    category_count = defaultdict(int)
    for entity in entities:
        try:
            attrs = json.loads(entity['attributes'])
            category = attrs.get('category', 'Unknown')
            category_count[category] += 1
        except:
            category_count['Unknown'] += 1
    
    print(f"\\nEntity categories:")
    for category, count in sorted(category_count.items(), key=lambda x: x[1], reverse=True):
        print(f"  • {category}: {count} entities")
    
    return G

# Visualize the subgraph
G = visualize_knowledge_graph(subgraph)